<a href="https://colab.research.google.com/github/antoniodragoc/Projeto-de-Graduacao-Antonio-Drago-Caetano/blob/main/3_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# import the needed packages
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import os

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
#from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# https://thedatafrog.com/en/articles/image-recognition-transfer-learning/

from keras.layers.core.flatten import Flatten
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Conv2D

## Referências

Artigo principal: https://arxiv.org/pdf/1810.09025.pdf

Configurações: https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/ 

Aumento de dados: https://pyimagesearch.com/2019/07/08/=keras-imagedatagenerator-and-data-augmentation/

Aplicação aumento de dados: https://gist.github.com/rstml/bbd491287efc24133b90d4f7f3663905

In [ ]:
# Gerando os dados a partir das imagens
train_datagen = ImageDataGenerator(    
    rotation_range = 90,
    vertical_flip = True,
    horizontal_flip = True,
    zoom_range=0.05,
    width_shift_range=0.2,
		height_shift_range=0.2,
    fill_mode = 'reflect',
    samplewise_center = True,
    samplewise_std_normalization = True
)

val_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True
    )


test_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True
    )

In [ ]:
# Caminhos de treino e validaçção

# Obs - no diretório dados4 os dados estão separados em 75% treino 15% validação e 10% teste
path_train = '/content/drive/MyDrive/PG/dados4/train/'
path_val = '/content/drive/MyDrive/PG/dados4/val/'

batch_size = 20

In [ ]:
# https://gist.github.com/rstml/bbd491287efc24133b90d4f7f3663905

In [ ]:
# dados de treino
train_generator = train_datagen.flow_from_directory(
    path_train,
    target_size = (299, 299),
    batch_size= batch_size,
    shuffle=True,
    class_mode='categorical' 
)

Found 300 images belonging to 4 classes.


In [ ]:
y_true = train_generator.classes
#y_true

In [ ]:
#dados de teste
val_generator = val_datagen.flow_from_directory(
    path_val,
    target_size = (299, 299),
    batch_size= batch_size,  
    shuffle=True,
    class_mode='categorical'     
)

Found 60 images belonging to 4 classes.


In [ ]:
val_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [ ]:
# Chamando a rede pré-treinada
conv_model = VGG19(
    weights='imagenet', 
    include_top=True, 
    )
conv_model.summary()

574726144/574710816 [==============================] - 3s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [ ]:
# Chamando a rede pré-treinada
conv_model = ResNet50(
    weights='imagenet', 
    include_top=True, 
    )
conv_model.summary()

102981632/102967424 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                        

In [ ]:
# Chamando a rede pré-treinada
conv_model = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(299,299,3))

first_freeze_layers = 20 #number of the first layers to freeze
for layer in conv_model.layers[0:first_freeze_layers]:
    layer.trainable = False

teste = keras.layers.Flatten()(conv_model.output)

# three hidden layers
teste = keras.layers.Dropout(0.4)(teste)
teste = keras.layers.Dense(128, activation='relu')(teste)
teste = keras.layers.Dense(64, activation='relu')(teste)

# final softmax layer with four categories
predictions = keras.layers.Dense(4, activation='softmax')(teste)

# creating the full model:
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)

full_model.summary()

94781440/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 305, 305, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                             

In [ ]:
#EARLY STOPPING ADICIONADO
# https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
callback = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=40,
    verbose=0,
    mode="min",
    baseline=None,
    restore_best_weights=True
)

lr_schedule = keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate = 0.01, 
    first_decay_steps = 500, 
    t_mul=2.4, 
    m_mul=1.3, 
    alpha=0.001, 
    name=None
)

In [ ]:
# Compilando o modelo
full_model.compile(loss='categorical_crossentropy',
                  optimizer= keras.optimizers.SGD(learning_rate=lr_schedule),
                  metrics=['accuracy'])

# Treinando o modelo com os dados de treino e teste
history = full_model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 200,
    callbacks=[callback]
)

Epoch 1/200
15/15 [==============================] - 20s 750ms/step - loss: 9.5775 - accuracy: 0.3100 - val_loss: 1.4089 - val_accuracy: 0.2500
Epoch 2/200
15/15 [==============================] - 9s 603ms/step - loss: 1.3609 - accuracy: 0.3300 - val_loss: 1.4377 - val_accuracy: 0.2667
Epoch 3/200
15/15 [==============================] - 10s 620ms/step - loss: 1.3465 - accuracy: 0.3467 - val_loss: 1.3779 - val_accuracy: 0.3000
Epoch 4/200
15/15 [==============================] - 9s 611ms/step - loss: 1.3159 - accuracy: 0.4433 - val_loss: 1.3816 - val_accuracy: 0.2833
Epoch 5/200
15/15 [==============================] - 10s 625ms/step - loss: 1.2121 - accuracy: 0.4800 - val_loss: 1.3431 - val_accuracy: 0.3333
Epoch 6/200
15/15 [==============================] - 9s 611ms/step - loss: 1.2401 - accuracy: 0.4600 - val_loss: 1.3568 - val_accuracy: 0.3000
Epoch 7/200
15/15 [==============================] - 9s 609ms/step - loss: 1.2100 - accuracy: 0.4533 - val_loss: 1.5205 - val_accuracy: 0.2

In [ ]:
#dados de teste
path_test = '/content/drive/MyDrive/PG/dados4/test/'
test_generator =  test_datagen.flow_from_directory(
    path_test,
    target_size = (299, 299),
    shuffle = False,
    batch_size= batch_size,
    class_mode='categorical' 
    
    )

Found 40 images belonging to 4 classes.


In [ ]:
#Confution Matrix and Classification Report
Y_pred = full_model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_generator.classes, y_pred )

array([[9, 1, 0, 0],
       [1, 9, 0, 0],
       [2, 1, 7, 0],
       [0, 1, 0, 9]])

In [ ]:
np.argmax(Y_pred, axis=1)


array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2,
       2, 2, 0, 2, 2, 2, 1, 2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3])

In [ ]:
test_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [ ]:
# Salvando a REDE RESNET

In [ ]:
checkpoint_path = '/content/drive/MyDrive/PG/codigo/checkpoint/'

checkpoint_dir = os.path.dirname(checkpoint_path)
#os.listdir(checkpoint_dir)

# Save the weights using the `checkpoint_path` format
full_model.save_weights(checkpoint_path.format(epoch=86))
full_model.load_weights(checkpoint_path)
full_model.save(checkpoint_path+'resnet0531')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/PG/codigo/checkpoint/resnet0531/assets
